In [60]:
import pandas as pd
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import time
from tqdm import tqdm

### 6개월치 proba 평균

In [15]:
df_weather=pd.read_csv('기상데이터 0510 index X.csv')
df_weather.head()

,관측일자,temperature,temp_max,temp_min,rainfall,humidity,조사지역,연,월,일
0,1988-01-01,-2.4,-0.8,-4.4,0.5,64.0,서울,1988,1,1
1,1988-01-01,-2.6,0.5,-6.2,1.9,73.0,관악산,1988,1,1
2,1988-01-01,-4.1,-0.7,-8.6,0.0,66.0,춘천,1988,1,1
3,1988-01-01,-3.1,-1.0,-8.6,0.4,79.0,철원,1988,1,1
4,1988-01-01,3.7,6.5,0.1,0.0,57.0,속초,1988,1,1


In [5]:
df_grain=pd.read_csv('작물추천_min_max_추가_데이터타입변경(코코아,밀 추가).csv')
del df_grain['Unnamed: 0']
df_grain.head()

,temperature,rainfall,temp_min,temp_max,label
0,20.879744,202.935536,18,28,rice
1,21.770462,226.655537,18,28,rice
2,23.004459,263.964248,18,28,rice
3,26.491096,242.864034,18,28,rice
4,20.130175,262.717340,18,28,rice


In [9]:
df_grain['label'].value_counts()

wheat          101
cocoa          101
muskmelon      100
coffee         100
cotton         100
coconut        100
papaya         100
orange         100
apple          100
rice           100
maize          100
grapes         100
mango          100
banana         100
pomegranate    100
lentil         100
kidneybeans    100
chickpea       100
watermelon     100
Name: label, dtype: int64

In [62]:
# 6개월치 데이터를 추합 return 습도 x
def predict_6m_proba(region, start_month, pred_year):
    df = pd.DataFrame()
    for mon in range(start_month, start_month+6):
        pred_data = OLS_model_predict(region, mon, pred_year)
        # print(pred_data)
        tmp_data =[]
        for data in pred_data :
            tmp_data.append(data[-1])
        # print(tmp_data)
        df_pred  = predict_crops_6m(tmp_data)
        df = pd.concat([df, df_pred])
    return df.groupby(df.index).mean().sort_values(by='확률', ascending=False)

def predict_crops_6m(data):
    tmp_grain = df_grain[['temperature', 'temp_max', 'temp_min',  'rainfall', 'label']]
    xgb = XGBClassifier()  
    labelencoder = LabelEncoder()
    target_lb = labelencoder.fit_transform(tmp_grain.label)
    scaler = MinMaxScaler()
    grain_data = tmp_grain[tmp_grain.columns[:-1]]

    grain_data= grain_data.append({'temperature' :  data[0], 'temp_max' : data[1], 'temp_min' : data[2], 'rainfall' : data[3]*30} , ignore_index=True)

    grain_mmx = scaler.fit_transform(grain_data)

    # display(grain_data.tail())
    # display(grain_mmx[:-1])
    # display(grain_mmx[-1])

    X_train, X_test, y_train, y_test = train_test_split(
        grain_mmx[:-1], target_lb, stratify=target_lb, test_size=0.2,
        random_state=2022
    )
    xgb.fit(X_train, y_train)

    # print(grain_mmx[-1])
    df_pred = pd.DataFrame(xgb.predict_proba(list(grain_mmx[-1])), columns=labelencoder.classes_).T
    df_pred.columns = ['확률']
    df_pred = df_pred.sort_values(by='확률',ascending=False)
    return_pred = df_pred
    return return_pred

def OLS_model_predict(region, month, pred_year):
    df_tmp_region = df_weather[df_weather['조사지역'] == region]
    df_weather_region = df_tmp_region.groupby(['조사지역', '관측일자']).mean()
    df_weather_region = df_weather_region.reset_index(drop=False)

    df_group_mean = df_weather_region.groupby(['연', '월']).mean()
    df_group_mean = df_group_mean.reset_index(drop=False)
    df_month_mean = df_group_mean[df_group_mean['월'] == month]

    for i in df_month_mean.index:
        df_tmp = df_weather_region.loc[(df_weather_region['연'] == df_month_mean.loc[i, '연']) & (df_weather_region['월'] == df_month_mean.loc[i, '월'])]
        temp_max = df_tmp['temp_max'].max()
        temp_min = df_tmp['temp_min'].min()
        df_month_mean.loc[i, 'temp_max'] = temp_max
        df_month_mean.loc[i, 'temp_min'] = temp_min

    pred_list = []
    for value in df_month_mean.columns[2: -1]:
        df_target = df_month_mean[value]
        X_train2 = pd.DataFrame(df_month_mean[['연']], columns = ['연'])
        y_train2 = df_target.values

        # import statsmodels.api as sm
        X_train = sm.add_constant(X_train2)
        model = sm.OLS(y_train2, X_train2).fit()
        pred_value = model.predict(pred_year)
        pred_list.append([value, float(pred_value)])

    # print(f'\t\t\t{region}의 {month}월 {pred_year}의 예측값:')
    # print(pred_list)
    return pred_list






In [31]:
predict_6m_proba('서울', 3, 2022)

,확률
cocoa,0.252553
wheat,0.180007
maize,0.173578
kidneybeans,0.071278
grapes,0.067793
rice,0.065942
lentil,0.046039
banana,0.025013
coconut,0.024333
coffee,0.016077


In [25]:
df.groupby(df.index).mean().sort_values(by='확률', ascending=False).head(3)

,확률
cocoa,0.252553
wheat,0.180007
maize,0.173578


In [63]:
for region in tqdm(df_weather['조사지역'].unique()) :
    print(f'{region}지역 추천 Top 3 작물')
    display(predict_6m_proba(region, 3, 2022).head(3))
    print('\n')

  0%|          | 0/73 [00:00<?, ?it/s]

서울지역 추천 Top 3 작물


,확률
cocoa,0.252553
wheat,0.180007
maize,0.173578


  1%|▏         | 1/73 [00:04<05:43,  4.77s/it]



관악산지역 추천 Top 3 작물


...

,확률
cocoa,0.301050
wheat,0.254634
rice,0.071254


 99%|█████████▊| 72/73 [05:52<00:04,  4.56s/it]



고창지역 추천 Top 3 작물


,확률
wheat,0.282840
cocoa,0.183927
maize,0.170499


100%|██████████| 73/73 [05:57<00:00,  4.89s/it]

In [64]:
# 모든지역 데이터 return
def predict_6m_all_region(start_month, pred_year):
    df = pd.DataFrame()
    for mon in range(start_month, start_month+6):
        region_data = []
        for region in df_weather['조사지역'].unique():
            pred_data = OLS_model_predict(region, mon, pred_year)
            tmp_data =[]
            for data in pred_data :
                tmp_data.append(data[-1])
            tmp_data.append(region)
            region_data.append(tmp_data)

        # print(tmp_data)
        df_pred  = predict_crops_6m(region_data)
        df = pd.concat([df, df_pred])
        df_gr = df.groupby(df.index).mean()
        for col in df_gr.columns :
            df_gr = df_gr.sort_values(by=col, ascending=False)
        
    return df_gr

def predict_crops_6m(data):
    tmp_grain = df_grain[['temperature', 'temp_max', 'temp_min',  'rainfall', 'label']]
    xgb = XGBClassifier()  
    labelencoder = LabelEncoder()
    target_lb = labelencoder.fit_transform(tmp_grain.label)
    scaler = MinMaxScaler()
    grain_data = tmp_grain[tmp_grain.columns[:-1]]

    for value in data:
        grain_data= grain_data.append({'temperature' :  value[0], 'temp_max' : value[1], 'temp_min' : value[2], 'rainfall' : value[3]*30} , ignore_index=True)

    grain_mmx = scaler.fit_transform(grain_data)

    # display(grain_data.tail())
    # display(grain_mmx[:-1])
    # display(grain_mmx[-1])

    X_train, X_test, y_train, y_test = train_test_split(
        grain_mmx[:-len(data)], target_lb, stratify=target_lb, test_size=0.2,
        random_state=2022
    )
    xgb.fit(X_train, y_train)

    # print(grain_mmx[-1])
    return_pred = pd.DataFrame()
    for i, value in enumerate(data):
        df_pred = pd.DataFrame(xgb.predict_proba(list(grain_mmx[-(len(data)-i)])), columns=labelencoder.classes_).T
        df_pred.columns = [value[4]]
        # df_pred = df_pred.sort_values(by='확률',ascending=False)
        return_pred = pd.concat([return_pred, df_pred], axis=1)
    return return_pred

def OLS_model_predict(region, month, pred_year):
    df_tmp_region = df_weather[df_weather['조사지역'] == region]
    df_weather_region = df_tmp_region.groupby(['조사지역', '관측일자']).mean()
    df_weather_region = df_weather_region.reset_index(drop=False)

    df_group_mean = df_weather_region.groupby(['연', '월']).mean()
    df_group_mean = df_group_mean.reset_index(drop=False)
    df_month_mean = df_group_mean[df_group_mean['월'] == month]

    for i in df_month_mean.index:
        df_tmp = df_weather_region.loc[(df_weather_region['연'] == df_month_mean.loc[i, '연']) & (df_weather_region['월'] == df_month_mean.loc[i, '월'])]
        temp_max = df_tmp['temp_max'].max()
        temp_min = df_tmp['temp_min'].min()
        df_month_mean.loc[i, 'temp_max'] = temp_max
        df_month_mean.loc[i, 'temp_min'] = temp_min

    pred_list = []
    for value in df_month_mean.columns[2: -2]:
        df_target = df_month_mean[value]
        X_train2 = pd.DataFrame(df_month_mean[['연']], columns = ['연'])
        y_train2 = df_target.values

        # import statsmodels.api as sm
        X_train = sm.add_constant(X_train2)
        model = sm.OLS(y_train2, X_train2).fit()
        pred_value = model.predict(pred_year)
        pred_list.append([region, float(pred_value)])

    # print(f'\t\t\t{region}의 {month}월 {pred_year}의 예측값:')
    # print(pred_list)
    return pred_list






In [66]:
%time predict_6m_all_region(3, 2022)

CPU times: user 1min 4s, sys: 771 ms, total: 1min 5s
Wall time: 1min 5s


,서울,관악산,춘천,철원,속초,원주,대관령,태백,인제,홍천,...,포항,울릉도,동해,영월,흑산도,동두천,문산,진도,상주,고창
wheat,0.180007,0.205366,0.254331,0.310365,0.264874,0.279594,0.355426,0.391814,0.378621,0.387507,...,0.319658,0.272166,0.341935,0.389324,0.094537,0.254451,0.191440,0.229759,0.254634,0.282840
cocoa,0.252553,0.169566,0.256804,0.278495,0.196778,0.254787,0.000238,0.190647,0.107870,0.054799,...,0.137461,0.333348,0.198751,0.094071,0.016196,0.226546,0.224886,0.334328,0.301050,0.183927
maize,0.173578,0.072840,0.199499,0.052169,0.174520,0.172975,0.303502,0.201363,0.061021,0.100162,...,0.061787,0.011425,0.174514,0.093990,0.060757,0.202497,0.174074,0.037832,0.054664,0.170499
coconut,0.024333,0.016910,0.019363,0.015535,0.022002,0.022734,0.000631,0.012159,0.026530,0.033808,...,0.183348,0.006459,0.022121,0.031110,0.201564,0.020409,0.020635,0.034410,0.022330,0.098260
banana,0.025013,0.008211,0.019218,0.015218,0.021791,0.020651,0.000246,0.004125,0.026520,0.035846,...,0.033261,0.007029,0.021916,0.036140,0.016644,0.020232,0.020504,0.011340,0.023317,0.091811
lentil,0.046039,0.008057,0.067388,0.040735,0.007382,0.067892,0.000239,0.003304,0.039858,0.072072,...,0.011216,0.006706,0.007448,0.071306,0.041601,0.065371,0.026006,0.010845,0.067833,0.059626
rice,0.065942,0.197775,0.068492,0.067250,0.097356,0.067433,0.190997,0.057510,0.113482,0.115400,...,0.024801,0.007343,0.096349,0.100693,0.076404,0.090060,0.090276,0.015647,0.071254,0.034296
coffee,0.016077,0.012377,0.016620,0.013142,0.016820,0.016321,0.000540,0.008131,0.021459,0.026762,...,0.022795,0.010554,0.015423,0.024567,0.044316,0.017205,0.014790,0.063045,0.017129,0.008177
mango,0.006889,0.007201,0.007117,0.005894,0.007939,0.006989,0.000213,0.003534,0.010329,0.016400,...,0.015686,0.006018,0.008007,0.010682,0.014506,0.007412,0.007645,0.009707,0.010199,0.007906
papaya,0.011724,0.010906,0.011960,0.009846,0.013406,0.012502,0.000352,0.006323,0.017163,0.018862,...,0.017469,0.008692,0.012864,0.017744,0.022536,0.012491,0.012154,0.017447,0.015037,0.007059
